In [3]:
import pandas as pd
import glob, os 

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import warnings
warnings.simplefilter('ignore')

In [6]:
path = 'drive-download-20181116T203707Z-001/50M_250_1000_1_final.csv'

In [7]:
df = pd.read_csv(path,index_col=None, header=0, lineterminator='\n')

In [8]:
processedDescriptionDF = df['user_description_text'].dropna()

In [9]:
%%time
#strip links
import re
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

processedDescriptionDF = processedDescriptionDF.apply(strip_links)

CPU times: user 2.69 s, sys: 10.1 ms, total: 2.7 s
Wall time: 2.7 s


In [10]:
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "i'm": "i am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    text = c_re.sub(replace, text)
    return text

processedDescriptionDF = processedDescriptionDF.apply(expandContractions)

In [11]:
%%time
#Remove Mentions
import string
def strip_mentions(text):
    entity_prefixes = ['@']
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

processedDescriptionDF = processedDescriptionDF.apply(strip_mentions)

CPU times: user 4.69 s, sys: 107 ms, total: 4.8 s
Wall time: 4.81 s


In [12]:
%%time
#Remove Hashtags
def strip_hashtags(text):
    entity_prefixes = ['#']
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

processedDescriptionDF = processedDescriptionDF.apply(strip_hashtags)

CPU times: user 4.5 s, sys: 107 ms, total: 4.6 s
Wall time: 4.6 s


In [13]:
%%time
#Remove Special characters
import unicodedata
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

processedDescriptionDF = processedDescriptionDF.apply(remove_special_characters)

CPU times: user 4.47 s, sys: 78.6 ms, total: 4.55 s
Wall time: 4.55 s


In [14]:
%%time
#Remove unjoin emojis
import string
def seperate_emoji(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

processedDescriptionDF = processedDescriptionDF.apply(remove_special_characters)

CPU times: user 3.04 s, sys: 85.8 ms, total: 3.13 s
Wall time: 3.13 s


In [148]:
processedDescriptionDF = processedDescriptionDF.apply(lambda x: x.lower())

In [16]:
df['processed_user_description'] = processedDescriptionDF

In [27]:
#%time df.to_csv('50M_250_1000_1_final_updated_desc.csv', index=False)

CPU times: user 1min 24s, sys: 2.35 s, total: 1min 27s
Wall time: 1min 28s
